<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/ExemplosGeracaoTexto_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Geração de textos usando Llama v2.0 usando Transformers by HuggingFace

Exemplo de uso do modelo de linguagem grande Llama v2.0.
- Analise da geração de textos
- Prompts com textos emparelhados
- Injentando padrões no prompt
- Padrão Persona

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Notebook de referência:**

https://github.com/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb


**Lista dos modelos:**

https://huggingface.co/models


**Artigos referências:**

https://dev.to/nithinibhandari1999/how-to-run-llama-2-on-your-local-computer-42g1


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

In [ ]:
!pip install bitsandbytes==0.41.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00


Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

In [ ]:
# accelerate: treino distribuído, mixed precision, consumer hardware
!pip install accelerate==0.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.1 MB/s eta 0:00:00


Instala a interface pytorch para o BERT by Hugging Face.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00


Instala o cliente do huggingface hub para realizar o login.

In [ ]:
!pip install huggingface_hub==0.18.0

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

In [ ]:
from huggingface_hub.hf_api import HfFolder

ACCESS_TOKEN  = 'COLOQUE O TOKEN DE ACESSO AQUI'

HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
!huggingface-cli whoami

osmarbraz


## 2.2 - Nome do modelo de linguagem

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

# Não carrega por falta de memória no google colab
#nome_modelo = "meta-llama/Llama-2-13b-hf"
#nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não carrega por falta de memória e espaço em disco no google colab
#nome_modelo = "meta-llama/Llama-2-70b-hf"
#nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## 2.4 - Carregando o Modelo LLM

Carregando o **modelo** da comunidade.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import AutoModelForCausalLM
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True,   # Carrega de um repositório confiável
                                             load_in_8bit=True,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

print("Tempo de carregamento do modelo:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...


(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Tempo de carregamento do modelo:  0:03:28 (h:mm:ss)


In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}



# 3 - Analisando a geração de textos



## 3.1 - Geração de texto


In [ ]:
# Define o documento base
documento = "Como empilhar elementos em uma pilha?"
#documento = "How to push elements in a stack"
#documento = "O comando SQL para extrair todos os usuários cujo nome começa com A é:"
#documento = "Bom dia professor, tudo bem ?"
# documento = "The SQL command to extract all the users whose name starts with A is:"
#documento = "How to push elements in a stack"
#documento = "Write code for finding the prime number in python ?"
# documento = "Escrever código para encontrar o número primo em python?"

# Prepara o prompt para enviar ao modelo realizando sua tokenização
# Se pt for especificado, ele retornará tensores em vez de lista de inteiros python e tokenizará os documentos
input = tokenizer(documento, return_tensors="pt")

# Mostra os tokens com seus índices
i = 0
for tup in input.input_ids[0]:
    # print(tup.item())
    print("{} {}".format(i, tokenizer.convert_ids_to_tokens(tup.item())))
    i= i + 1

0 <s>
1 ▁Como
2 ▁emp
3 il
4 har
5 ▁elementos
6 ▁em
7 ▁uma
8 ▁pil
9 ha
10 ?


In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

# Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
input_ids = input["input_ids"].to(model.device)

# Envia a prompt preparado ao modelo
outputs = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

# Liberar memória
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print(len(outputs))

4


In [ ]:
print(outputs)

BeamSearchDecoderOnlyOutput(sequences=tensor([[    1, 17295,  3710,   309,  8222, 29290,   953,  3672,  8230,  2350,
         29973,    13,    13,  1523, 29877,  3710,   309,  8222, 29290,   953,
          3672,  8230,  2350, 29973,   319,  6578,   707,  1368,  3093,   398,
           294,   270,  5070,  1702,  7931, 30037,  3710,   309,  8222, 29290,
           953,  3672,  8230,  2350,   316,  5954,   321,  9639,  6759, 29901,
            13,    13, 29896, 29889, 16760,   346,   419,  1922,  1543, 29877,
          6815, 29901, 16760,   346,   263,  3710,   309,  8222,  2897, 29290,
          1146,  8230,  2350,   419,  1922,  1543, 29877,  6815, 29892,  1986,
          1922,  7294,   307,  2123,  3672,  5777, 19892, 29889,  1317,   578,
         13612,   566, 20484,   263,  4016,   309, 15356,   263,  2967,  1146,
          8230,  2350, 29889,    13, 29906, 29889,  4803, 29290, 29871,  5526,
          1759, 29901,  4803, 29290, 29871,  5526,  1759,   953,   260, 13533,
          1251

Decodificação

Nossa etapa de geração gera uma matriz de tokens em vez de palavras. Para converter esses tokens em palavras, precisamos realizar sua decodificação.

In [ ]:
# Mostra o resultado
for s in outputs.sequences:
  # Decodifica a saída
  # skip_special_tokens=True retira os tokens especiais da saída da decodificação
  output = tokenizer.decode(s, skip_special_tokens=True)
  print(output)

Como empilhar elementos em uma pilha?

Como empilhar elementos em uma pilha? Aqui estão algumas dicas para você empilhar elementos em uma pilha de forma eficiente:

1. Comece com um elemento grande: Comece a empilhar os elementos da pilha com um elemento grande, como um livro ou uma caixa. Isso ajudará a estabilizar a base da pilha.
2. Use elementos iguais: Use elementos iguais em tamanho e peso para garantir que a pilha seja estável. Se você estiver empilhando livros, por exemplo, use livros de tamanho e peso semelhantes.
3. Aumente a altura gradualmente: Aumente a altura da pilha gradualmente, adicionando elementos de tamanho cada vez maior. Isso ajudará a garantir que a pilha seja estável e não deslize.
4. Use uma superfície plana: Use uma superfície plana para empilhar os elementos, como uma mesa ou um chão plano. Isso ajud


In [ ]:
# Mostra o resultado
# skip_special_tokens=True retira os tokens especiais da saída da decodificação
print(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))

Como empilhar elementos em uma pilha?

Como empilhar elementos em uma pilha? Aqui estão algumas dicas para você empilhar elementos em uma pilha de forma eficiente:

1. Comece com um elemento grande: Comece a empilhar os elementos da pilha com um elemento grande, como um livro ou uma caixa. Isso ajudará a estabilizar a base da pilha.
2. Use elementos iguais: Use elementos iguais em tamanho e peso para garantir que a pilha seja estável. Se você estiver empilhando livros, por exemplo, use livros de tamanho e peso semelhantes.
3. Aumente a altura gradualmente: Aumente a altura da pilha gradualmente, adicionando elementos de tamanho cada vez maior. Isso ajudará a garantir que a pilha seja estável e não deslize.
4. Use uma superfície plana: Use uma superfície plana para empilhar os elementos, como uma mesa ou um chão plano. Isso ajud


# 4 - Exemplos de textos emparelhados

In [ ]:
def gerar_prompt(texto, entrada=None):
    if entrada:
        return f"""Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Entrada:
{entrada}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto, entrada=None):
    # Recupera o prompt
    prompt = gerar_prompt(texto, entrada)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

## 4.1 - Tarefa simples

In [ ]:
texto = 'Me fale sobre algoritmos.'
avaliarTexto(texto)

Resposta: 
Algoritmos são sequências de estágios de resolução de problemas que são usados para resolver problemas complexos em computação. Eles são usados em uma variedade de aplicações, como processamento de linguagem natural, visão computacional, inteligência artificial e análise de dados. Algoritmos podem ser clasificados em diferentes categorias, como algoritmos de busca, algoritmos de processamento de linguagem, algoritmos de aprendizado de máquina e algoritmos de análise de dados. Cada tipo de algoritmo tem suas próprias características e é usado para resolver problemas específicos. Alguns exemplos de algoritmos incluem o algoritmo de busca binária, o algoritmo de reorganização de dicionário, o algoritmo de aprendizado de máquina de rede neuronal e o algoritmo de análise de dados de clustering.


## 4.2 - Tarefa com entrada

In [ ]:
texto = 'Dada a fórmula química, calcule a massa molar.'

entrada = 'CaCl2'

avaliarTexto(texto, entrada)

Resposta: 
A massa molar de CaCl2 é de aproximadamente 105,5 g/mol.

Raisei! 😃


In [ ]:
texto = 'Faça quatro perguntas sobre a seguinte passagem:'

entrada = 'A anatomia de uma abelha é bastante intrincada. Tem três partes do corpo: a cabeça, o tórax e o abdômen. A cabeça consiste em órgãos sensoriais, três olhos simples e dois olhos compostos e vários apêndices. O tórax tem três pares de pernas e dois pares de asas, enquanto o abdômen contém a maioria dos órgãos da abelha, incluindo o sistema reprodutivo e o sistema digestivo.'

avaliarTexto(texto, entrada)

Resposta: 
Espero que essas perguntas ajudem você a entender melhor a anatomia da abelha:

1. Qual é o nome da parte do corpo da abelha que contém os órgãos sensoriais?
2. Qual é o nome das pernas do tórax da abelha?
3. Qual é o nome do sistema reprodutivo da abelha?
4. Qual é o nome do sistema digestivo da abelha?


In [ ]:
texto = 'Analise o documento jurídico fornecido e explique os pontos-chave.'

entrada = 'O seguinte é um trecho de um contrato entre duas partes, rotulado como "Empresa A" e "Empresa B": "A Empresa A concorda em fornecer assistência razoável à Empresa B para garantir a precisão das demonstrações financeiras que fornece. Isso inclui permitir à Empresa um acesso razoável ao pessoal e outros documentos que possam ser necessários para a revisão da Empresa B. A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A".'

avaliarTexto(texto, entrada)

Resposta: 
O documento jurídico fornecido é um contrato entre duas partes, em que a Empresa A concorda em fornecer assistência para garantir a precisão das demonstrações financeiras da Empresa B. A Empresa B, por sua vez, concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A. Os pontos-chave deste contrato são:

* A Empresa A concorda em fornecer assistência para garantir a precisão das demonstrações financeiras da Empresa B.
* A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A.
* A Empresa A e a Empresa B estão de acordo em mantener o documento em confiança e proteger as informações confidenciais da Empresa A.

Essa é a conclusão que eu cheguei ao analisar o documento jurídico fornecido.


# 5 - Exemplos de injeção de padrões em prompts

 A injeção de padrões faz ignora filtros ou manipula o LLM usando prompts cuidadosamente elaborados que fazem o modelo ignorar instruções anteriores ou executar ações não intencionais.

 https://medium.com/@austin-stubbs/llm-security-types-of-prompt-injection-d7ad8d7d75a3


## 5.1 - Extração de Informação

In [ ]:
def gerar_promptEI(texto):
    ### texto:
    return f"""TEXTO: {texto}
Dado o texto acima, extraia informações importantes no formato abaixo:
<CHAVE>:<VALOR>
### Resposta: """

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarEI(texto):
    # Recupera o prompt
    prompt = gerar_promptEI(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

avaliarEI(texto)

Resposta: 
* Nome: Alan Mathison Turing
* Data de nascimento: 23 de junho de 1912
* Localização de nascimento: Londres
* Data de falecimento: 7 de junho de 1954
* Localização de falecimento: Wilmslow, Cheshire
* Realizações: matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico
* Contribuições: formalização dos conceitos de algoritmo e computação com a máquina de Turing, considerada um modelo de um computador de uso geral
* Reconhecimento: não foi totalmente reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.


## 5.2 - Entidade nomeada

In [ ]:
def gerar_promptEN(texto):
    ### texto:
    return f"""Detecte as entidades nomeadas no texto a seguir delimitado por aspas triplas.
Retorne a resposta no formato json com spans( Um array que representa o intervalo de caracteres (índices) nos quais a entidade nomeada ocorre no texto original. O primeiro valor no array é o índice inicial e o segundo é o índice final) das entidades nomeadas com os campos \'entidadeNomeada\', \'tipo\', \'span\'.
Retorne todas as entidades
'''{texto}'''
arquivo no formato json:
### Resposta: """

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarEN(texto):
    # Recupera o prompt
    prompt = gerar_promptEN(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

avaliarEN(texto)

Resposta: 
```json
[
  {
    "entidadeNomeada": "Alan Mathison Turing",
    "tipo": "Pessoa",
    "span": [
      0,
      13
    ]
  }
]
```


## 5.3 - Análise de sentimentos

### 5.3.1 - Análise de sentimentos 1

In [ ]:
def gerar_promptAS1(texto):

  return f"""Classifique os exemplos a seguir de acordo com as seguintes polaridades Positivo, Negativo e Neutro.
EXEMPLO:\n {texto}
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarAS1(texto):
    # Recupera o prompt
    prompt = gerar_promptAS1(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS1(texto)

Resposta: 
1 - Positivo
2 - Negativo
3 - Negativo
4 - Negativo
5 - Positivo


### 5.3.2 - Análise de sentimentos 2

In [ ]:
def gerar_promptAS2(texto):

  return  f"""EXEMPLO: {texto}
Classifique os exemplos de declarações acima de acordo com as polaridades Positivo, Negativo e Neutro.
Utilize o seguinte formato:\n###DECLARAÇÃO:<DECLARAÇÃO>\n###POLARIDADE:<POLARIDADE>.
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarAS2(texto):
    # Recupera o prompt
    prompt = gerar_promptAS2(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS2(texto)

Resposta: 
DECLARAÇÃO: Minha Experiência na loja foi incrível.
POLARIDADE: Positivo.

DECLARAÇÃO: Eu acho que podiam melhorar o produto.
POLARIDADE: Negativo.

DECLARAÇÃO: O atendimento foi horrível!
POLARIDADE: Negativo.

DECLARAÇÃO: Não volto mais.
POLARIDADE: Negativo.

DECLARAÇÃO: Recomendo demais a banoffe. É uma delícia!
POLARIDADE: Positivo.


## 5.4 - Pergunta e resposta

In [ ]:
def gerar_promptPR(texto):
    '''
      Alterações no texto e tabulação impedem a geração da resposta.
    '''
    return f"""Dado o texto a seguir: {texto}\n
            Gere quatro questões em língua portuguesa e suas respectivas respostas utilizando o template abaixo.\n
            Preserve a exata formatação do template apresentado: \n
            PERGUNTA:<PERGUNTA>
            RESPOSTA:<RESPOSTA>
            ### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarPR(texto):
    # Recupera o prompt
    prompt = gerar_promptPR(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

# Tempo de processamento: ~2m20s
avaliarPR(texto)

Resposta: 
<RESPOSTA>

            PERGUNTA: Como Alan Turing é considerado o pai da ciência da computação teórica?
            RESPOSTA: Turing proporcionou uma formalização dos conceitos de algoritmo e computação com a máquina de Turing, que pode ser considerada um modelo de um computador de uso geral.

            PERGUNTA: Por que Alan Turing nunca foi totalmente reconhecido em seu país de origem durante sua vida?
            RESPOSTA: Ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.

            PERGUNTA: Qual é o significado da Lei de Segredos Oficiais para Alan Turing?
            RESPOSTA: A Lei de Segredos Oficiais foi uma lei britânica que proibia qualquer pessoa de discutir ou divulgar qualquer informação que pudesse prejudicar a segurança do país durante a Segunda Guerra Mundial.

            PERGUNTA: Em que ano morreu Alan Turing?
            RES

# 6 - Exemplos de padrão de pessoa (padrão persona) em prompts

## 6.1 Um matemático

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um professor de matemática. Me explique no idioma portuguesa a importância do teorema de pitágoras.'

avaliarTexto(texto)

Resposta: 
Bem-vindos à minha aula de matemática! Hoje, vamos abordar um dos teoremas mais importantes da história das matemáticas: o Teorema de Pitágoras.

O Teorema de Pitágoras é um dos principais resultados da geometria euclidiana, que estabelece uma relação matemática entre os lados de um triângulo retângulo. Em resumo, o teorema afirma que, se um triângulo retângulo tem um cateto maior e um cateto menor, então o quadrado da medida do cateto maior é igual à soma dos quadrados dos catetos menores.

Mas, você pode perguntar, por quê é importante esse teorema? E a resposta é simples: o Teorema de Pitágoras tem aplicações em diversas áreas da vida, desde a engenharia até a física e a informática.

Para começar, o Teorema de Pitágoras é fundamental na construção de edifícios e outras estruturas. Imagine que você é um arquiteto e precisa construir um prédio. Para isso, você precisa calcular a distância entre as bases e as alturas dos edifícios. O Teorema de Pitágoras é uma ferramenta fu

## 6.2 Um advogado

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um advogado brasileiro especialista em direito penal. '\
        'Pontue de forma resumida as possíveis penas para um caso de lesão corporal leve sem contexto de violência doméstica.'

avaliarTexto(texto)

Resposta: 
Com base nos artigos 129 e 130 do Código Penal Brasileiro, as penas para um caso de lesão corporal leve sem contexto de violência doméstica podem variar de acordo com a grau da lesão e a intenção do agente. Aqui estão algumas possíveis penas:

* Lesão leve: Penas de 1 (um) a 3 (três) meses de reclusão, multa de 100 a 300 reais ou ambas as penas.
* Lesão moderada: Penas de 3 (três) a 6 (seis) meses de reclusão, multa de 300 a 600 reais ou ambas as penas.
* Lesão grave: Penas de 6 (seis) a 12 (doze) meses de reclusão, multa de 600 a 1.200 reais ou ambas as penas.

Além disso, é importante considerar que, se a lesão for causada por um ato deliberado e intencional, pode haver a inclusão de penas adicionais, como a degradação do direito, a perda do direito de exercer determinadas atividades profissionais ou a ineptidão.

É importante lembrar que essas são apenas possíveis penas e que o juiz tem a discreção de aplicar penas mais severas ou mais lenientes, considerando os fatos e c

## 6.3 Um astrofísico

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um astrofísico. Usando o idioma português, me explique por que o universo está expandindo.'

avaliarTexto(texto)

Resposta: 
Olá! Como astrofísico, podemos entender a expansão do universo através de uma série de evidências e teoremas fundamentais da física.

Em primeiro lugar, é importante compreender que o universo é uma vastidão de matéria e energia que se estende em todas as direções. Ao longo do tempo, a matéria e a energia no universo se espalharam, e isso levou a uma expansão do universo em si.

A expansão do universo pode ser entendida como um processo de dilatação da distância entre as estrelas e galáxias. Essa distância aumentou ao longo do tempo, o que significa que as estrelas e galáxias estão se afastando uns das outras.

A evidência mais forte para a expansão do universo vem de observações de galáxias distantes. Em meados da década de 1920, o astrônomo Edwin Hubble descobriu que as galáxias distantes parecem mais fracas do que as galáxias próximas. Isso pode ser explicado se a distância entre as galáxias aumentar ao longo do tempo, o que significa que as galáxias estão se afastando un